In [1]:
import gc
import time
import model_img
import fastText
from keras.callbacks import ModelCheckpoint
from importlib import reload
from keras import backend as K

Using TensorFlow backend.


In [2]:
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf

%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=3

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3


In [14]:
reload(model_img)

<module 'model_img' from '/srv/home/gwiedemann/pss-lre/model_img.py'>

In [3]:
img_dim = (224,224)

In [4]:
data_train = model_img.read_csv_data("data/archive20k/text/dataset.train")
data_test = model_img.read_csv_data("data/archive20k/text/dataset.test")

In [32]:
model_singlepage = model_img.compile_model_singlepage(img_dim, print_summary=False)

Training from layer 17


In [8]:
# del model_singlepage
# model_singlepage.summary()

In [35]:
model_file = "exp1_single-page_repeat-%02d.hdf5" % (0,)
checkpoint = model_img.ValidationCheckpoint(model_file, data_test[:200], img_dim, metric='kappa')
model_singlepage.fit_generator(generator=model_img.ImageFeatureGenerator(data_train[:1000], img_dim),
                callbacks = [checkpoint],
                epochs = 5)

Epoch 1/5
32/32 [==============================] - 6s 190ms/step - loss: 2.6737 - acc: 0.7423

kappa improvement: -0.08193883439122929 (before: -inf), saving to exp1_single-page_repeat-00.hdf5
Epoch 2/5
32/32 [==============================] - 6s 199ms/step - loss: 2.2638 - acc: 0.7695

kappa improvement: -0.07667685144769387 (before: -0.08193883439122929), saving to exp1_single-page_repeat-00.hdf5
Epoch 3/5
32/32 [==============================] - 6s 189ms/step - loss: 2.0368 - acc: 0.7919

kappa improvement: 0.056102100505184826 (before: -0.07667685144769387), saving to exp1_single-page_repeat-00.hdf5
Epoch 4/5
32/32 [==============================] - 6s 189ms/step - loss: 1.6061 - acc: 0.8280

kappa improvement: 0.08042821850123527 (before: 0.056102100505184826), saving to exp1_single-page_repeat-00.hdf5
Epoch 5/5
32/32 [==============================] - 6s 191ms/step - loss: 1.3278 - acc: 0.8418

kappa improvement: 0.16312056737588654 (before: 0.08042821850123527), saving to exp1_s

In [36]:
import numpy as np
np.round(model_singlepage.predict_generator(generator=model_img.ImageFeatureGenerator(data_train[:10], img_dim)))

array([[0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)

In [38]:
reload(model_img)
model_slices = model_img.compile_model_singlepage_slices(img_dim, print_summary=False)

In [39]:
model_file = "exp1_slices_repeat-%02d.hdf5" % (0,)
checkpoint = model_img.ValidationCheckpoint(model_file, data_test[:200], img_dim, prev_page_generator='slices', metric='kappa')
model_slices.fit_generator(generator=model_img.ImageFeatureGenerator(data_train[:1000], img_dim, slices=True),
                callbacks = [checkpoint],
                epochs = 2)

Epoch 1/2
32/32 [==============================] - 10s 305ms/step - loss: 0.9415 - acc: 0.6651

kappa improvement: 0.14750198255352887 (before: -inf), saving to exp1_slices_repeat-00.hdf5
Epoch 2/2
32/32 [==============================] - 7s 228ms/step - loss: 0.4112 - acc: 0.7852

kappa improvement: 0.24184988627748305 (before: 0.14750198255352887), saving to exp1_slices_repeat-00.hdf5


In [40]:
import numpy as np
print(np.round(model_slices.predict_generator(generator=model_img.ImageFeatureGenerator(data_train[:10], img_dim, slices=True))))
model_slices.predict_generator(generator=model_img.ImageFeatureGenerator(data_train[:10], img_dim, slices=True))

[[1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]]


array([[0.5363179 ],
       [0.47066787],
       [0.49822107],
       [0.5670634 ],
       [0.56869125],
       [0.64747995],
       [0.6168937 ],
       [0.42107165],
       [0.5875879 ],
       [0.5881664 ]], dtype=float32)

# Experiment: VGG CNN model (single page)
* optimize for Kappa
* 10 repeats

In [ ]:
n_repeats = 10
n_epochs = 20
exp_history = []
optimize_for = 'kappa'
for i in range(n_repeats):
    print("Repeat " + str(i+1) + " of " + str(n_repeats))
    print("-------------------------")
    
    if 'model_singlepage' in locals():
        del model_singlepage
        del checkpoint
        gc.collect()
        time.sleep(2)
        K.clear_session()
        time.sleep(2)
    
    model_singlepage = model_img.compile_model_singlepage(img_dim)
    model_file = "exp2_img_repeat-%02d.hdf5" % (i,)
    checkpoint = model_img.ValidationCheckpoint(model_file, data_test, img_dim, metric=optimize_for)
    model_singlepage.fit_generator(
        generator=model_img.ImageFeatureGenerator(data_train, img_dim),
        callbacks = [checkpoint],
        epochs = n_epochs
    )
    
#     model_slices = model_img.compile_model_singlepage_slices(img_dim, print_summary=False)
#     model_file = "exp1_slices_repeat-%02d.hdf5" % (i,)
#     checkpoint = model_img.ValidationCheckpoint(model_file, data_test, img_dim, prev_page_generator='slices', metric=optimize_for)
#     model_slices.fit_generator(
#         generator=model_img.ImageFeatureGenerator(data_train, img_dim, slices = True),
#         callbacks = [checkpoint],
#         epochs = n_epochs
#     )
    

    with open("current_learning_progress.txt", "a") as pfile:
        pfile.write(str(i) + ": " + str(checkpoint.max_metrics) + '\n')

    exp_history.append(checkpoint.max_metrics)

avg_result = sum([m['kappa'] for m in exp_history]) / n_repeats
avg_acc = sum([m['accuracy'] for m in exp_history]) / n_repeats
print("-------------------------")
print(avg_result)
print(avg_acc)

Repeat 1 of 10
-------------------------
Training from layer 17
Epoch 1/20
543/543 [==============================] - 104s 192ms/step - loss: 1.9720 - acc: 0.7808

kappa improvement: 0.5136902495367197 (before: -inf), saving to exp2_img_repeat-00.hdf5
Epoch 2/20
543/543 [==============================] - 128s 235ms/step - loss: 1.1865 - acc: 0.8341
Epoch 3/20
543/543 [==============================] - 105s 193ms/step - loss: 0.8952 - acc: 0.8460

kappa improvement: 0.544432994593909 (before: 0.5136902495367197), saving to exp2_img_repeat-00.hdf5
Epoch 4/20
543/543 [==============================] - 103s 191ms/step - loss: 0.7020 - acc: 0.8625

kappa improvement: 0.5610624677468194 (before: 0.544432994593909), saving to exp2_img_repeat-00.hdf5
Epoch 5/20
543/543 [==============================] - 103s 190ms/step - loss: 0.6310 - acc: 0.8636
Epoch 6/20
543/543 [==============================] - 104s 191ms/step - loss: 0.5377 - acc: 0.8715
Epoch 7/20
543/543 [=============================

In [29]:
for i, r in enumerate(exp_history):
    model_file = "exp2_img_repeat-%02d.hdf5" % (i,)
    print(str(i) + ' ' + str(r['kappa']) + ' ' + str(r['accuracy']) + ' ' + model_file)
print("------------------------- VGG16 non-trainable")
print(avg_result)
print(avg_acc)

0 0.5122812735881425 0.8904808635917566 exp2_img_repeat-00.hdf5
1 0.4999373776599725 0.8777232580961727 exp2_img_repeat-01.hdf5
2 0.5141369577727235 0.8975466143277723 exp2_img_repeat-02.hdf5
3 0.5337580619138436 0.8914622178606477 exp2_img_repeat-03.hdf5
4 0.5230150789680554 0.8971540726202158 exp2_img_repeat-04.hdf5
5 0.512712131890368 0.8914622178606477 exp2_img_repeat-05.hdf5
6 0.5136244734338904 0.8896957801766437 exp2_img_repeat-06.hdf5
7 0.5009906416932111 0.8930323846908734 exp2_img_repeat-07.hdf5
8 0.4898900859723583 0.8781157998037291 exp2_img_repeat-08.hdf5
9 0.5111031958218717 0.8902845927379784 exp2_img_repeat-09.hdf5
------------------------- VGG16 non-trainable
0.5111449278714437
0.8896957801766439


In [32]:
for i, r in enumerate(exp_history):
    model_file = "exp2_img_repeat-%02d.hdf5" % (i,)
    print(str(i) + ' ' + str(r['kappa']) + ' ' + str(r['accuracy']) + ' ' + model_file)
print("------------------------- VGG16 13 Layers non-trainable")
print(avg_result)
print(avg_acc)

0 0.5531735085187636 0.8920510304219823 exp2_img_repeat-00.hdf5
1 0.5391492250163672 0.8894995093228656 exp2_img_repeat-01.hdf5
2 0.5136198298138486 0.8861629048086359 exp2_img_repeat-02.hdf5
3 0.5467867137526652 0.8951913640824337 exp2_img_repeat-03.hdf5
4 0.5521696484918528 0.8865554465161923 exp2_img_repeat-04.hdf5
5 0.5432006986659095 0.8977428851815505 exp2_img_repeat-05.hdf5
6 0.5576752420388299 0.8849852796859666 exp2_img_repeat-06.hdf5
7 0.5670913648399796 0.8997055937193327 exp2_img_repeat-07.hdf5
8 0.5544023782109659 0.8930323846908734 exp2_img_repeat-08.hdf5
9 0.5491288029050074 0.8861629048086359 exp2_img_repeat-09.hdf5
------------------------- VGG16 13 Layers non-trainable
0.547639741225419
0.8911089303238467


In [11]:
for i, r in enumerate(exp_history):
    model_file = "exp2_img_repeat-%02d.hdf5" % (i,)
    print(str(i) + ' ' + str(r['kappa']) + ' ' + str(r['accuracy']) + ' ' + str(r['f1_macro']) + ' ' + model_file)
print("------------------------- VGG16 13 Layers non-trainable")
print(avg_result)
print(avg_acc)

0 0.5574294841351988 0.8949950932286556 0.7786291816168562 exp2_img_repeat-00.hdf5
1 0.5786048536293394 0.8993130520117762 0.7892472291143108 exp2_img_repeat-01.hdf5
2 0.5701592616825089 0.898135426889107 0.7849915384739852 exp2_img_repeat-02.hdf5
3 0.545950299031571 0.8930323846908734 0.7728533041069929 exp2_img_repeat-03.hdf5
4 0.5399405782373612 0.8857703631010795 0.7699685046699973 exp2_img_repeat-04.hdf5
5 0.566172097361854 0.8951913640824337 0.7830610736105921 exp2_img_repeat-05.hdf5
6 0.5708232192518526 0.8971540726202158 0.785364961085953 exp2_img_repeat-06.hdf5
7 0.580795509810331 0.8987242394504417 0.7903736216911338 exp2_img_repeat-07.hdf5
8 0.5415033443224394 0.8979391560353287 0.770149525134159 exp2_img_repeat-08.hdf5
9 0.561660578334642 0.9010794896957802 0.7803972464222235 exp2_img_repeat-09.hdf5
------------------------- VGG16 13 Layers non-trainable
0.5613039225797098
0.8961334641805692


In [ ]:
from keras.applications.vgg16 import preprocess_input
image_file = '15_1_0280.png'
tiles = image_slicer.slice(image_file, 6, save = False)
slices = []
for tile in tiles:
    image = img_to_array(tile.image)
    print(image.shape)
    image = np.repeat(image, 3, axis=2)
    print(image.shape)
    slice = preprocess_input(image)
    slices.append(slice)

In [ ]:
print("-------------------------")
print(avg_result)
print(avg_acc)

In [ ]:
testlist = [[1,2,3,4],[5,6,7,8]]
np.array(list(zip(*testlist)))

# Experiment: Predecessor page

*IMPORTANT RESULT*: 
So far, it looks that predecessor image information does not contribute to PSS!

In [8]:
reload(model_img)

<module 'model_img' from '/srv/home/gwiedemann/pss-lre/model_img.py'>

In [15]:
n_repeats = 10
n_epochs = 15
exp2b_history = []
optimize_for = 'kappa'
for i in range(n_repeats):
    print("Repeat " + str(i+1) + " of " + str(n_repeats))
    print("-------------------------")
    
    if 'model_prevpage' in locals():
        del model_prevpage
        del checkpoint
        gc.collect()
        time.sleep(2)
        K.clear_session()
        time.sleep(2)
    
    model_prevpage = model_img.compile_model_prevpage(img_dim)
    model_file = "exp2_prev-page_repeat-%02d.hdf5" % (i,)
    checkpoint = model_img.ValidationCheckpoint(model_file, data_test, img_dim, prev_page_generator = True, metric=optimize_for)
    model_prevpage.fit_generator(generator=model_img.ImageFeatureGenerator(data_train, img_dim, prevpage=True),
                    callbacks = [checkpoint],
                    epochs = n_epochs)
    exp2b_history.append(checkpoint.max_metrics)
    
    with open("current_learning_progress.txt", "a") as pfile:
        pfile.write(str(i) + ": " + str(checkpoint.max_metrics) + '\n')

avg_kappa = sum([m['kappa'] for m in exp2b_history]) / n_repeats
avg_acc = sum([m['accuracy'] for m in exp2b_history]) / n_repeats
print("-------------------------")
print(avg_kappa)
print(avg_acc)

Repeat 1 of 10
-------------------------
Epoch 1/15
543/543 [==============================] - 120s 220ms/step - loss: 0.5579 - acc: 0.8246

kappa improvement: 0.49883967897177706 (before: -inf), saving to exp2_prev-page_repeat-00.hdf5
Epoch 2/15
543/543 [==============================] - 116s 213ms/step - loss: 0.2403 - acc: 0.8996

kappa improvement: 0.537993461608826 (before: 0.49883967897177706), saving to exp2_prev-page_repeat-00.hdf5
Epoch 3/15
543/543 [==============================] - 116s 214ms/step - loss: 0.1861 - acc: 0.9243
Epoch 4/15
543/543 [==============================] - 116s 214ms/step - loss: 0.1546 - acc: 0.9359

kappa improvement: 0.5685703637963293 (before: 0.537993461608826), saving to exp2_prev-page_repeat-00.hdf5
Epoch 5/15
543/543 [==============================] - 116s 214ms/step - loss: 0.1186 - acc: 0.9526
Epoch 6/15
543/543 [==============================] - 117s 215ms/step - loss: 0.0969 - acc: 0.9620
Epoch 7/15
543/543 [==============================] 

Epoch 15/15
543/543 [==============================] - 117s 215ms/step - loss: 0.0375 - acc: 0.9862
Repeat 5 of 10
-------------------------
Epoch 1/15
543/543 [==============================] - 117s 216ms/step - loss: 0.5808 - acc: 0.8226

kappa improvement: 0.3269189343705814 (before: -inf), saving to exp2_prev-page_repeat-04.hdf5
Epoch 2/15
543/543 [==============================] - 117s 215ms/step - loss: 0.2493 - acc: 0.8942

kappa improvement: 0.483522150305963 (before: 0.3269189343705814), saving to exp2_prev-page_repeat-04.hdf5
Epoch 3/15
543/543 [==============================] - 116s 213ms/step - loss: 0.1931 - acc: 0.9213

kappa improvement: 0.5460484804286824 (before: 0.483522150305963), saving to exp2_prev-page_repeat-04.hdf5
Epoch 4/15
543/543 [==============================] - 116s 213ms/step - loss: 0.1570 - acc: 0.9373
Epoch 5/15
543/543 [==============================] - 116s 214ms/step - loss: 0.1236 - acc: 0.9499

kappa improvement: 0.5693049287324634 (before: 0.546

543/543 [==============================] - 116s 213ms/step - loss: 0.0514 - acc: 0.9817
Epoch 12/15
543/543 [==============================] - 116s 214ms/step - loss: 0.0367 - acc: 0.9871

kappa improvement: 0.5853219844055793 (before: 0.5794326442410999), saving to exp2_prev-page_repeat-07.hdf5
Epoch 13/15
543/543 [==============================] - 116s 214ms/step - loss: 0.0349 - acc: 0.9873
Epoch 14/15
543/543 [==============================] - 116s 214ms/step - loss: 0.0327 - acc: 0.9892
Epoch 15/15
543/543 [==============================] - 116s 214ms/step - loss: 0.0342 - acc: 0.9879
Repeat 9 of 10
-------------------------
Epoch 1/15
543/543 [==============================] - 117s 216ms/step - loss: 0.6227 - acc: 0.8162

kappa improvement: 0.3709934150876003 (before: -inf), saving to exp2_prev-page_repeat-08.hdf5
Epoch 2/15
171/543 [========>.....................] - ETA: 1:20 - loss: 0.2659 - acc: 0.8845

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



543/543 [==============================] - 116s 214ms/step - loss: 0.0593 - acc: 0.9780

kappa improvement: 0.5584571725000762 (before: 0.5564292352087646), saving to exp2_prev-page_repeat-08.hdf5
Epoch 10/15
543/543 [==============================] - 116s 214ms/step - loss: 0.0526 - acc: 0.9804

kappa improvement: 0.5805844652202035 (before: 0.5584571725000762), saving to exp2_prev-page_repeat-08.hdf5
Epoch 11/15
543/543 [==============================] - 116s 213ms/step - loss: 0.0461 - acc: 0.9827
Epoch 12/15
543/543 [==============================] - 116s 213ms/step - loss: 0.0376 - acc: 0.9876
Epoch 13/15
543/543 [==============================] - 117s 216ms/step - loss: 0.0377 - acc: 0.9855
Epoch 14/15
357/543 [==================>...........] - ETA: 39s - loss: 0.0374 - acc: 0.9867

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



543/543 [==============================] - 116s 214ms/step - loss: 0.0749 - acc: 0.9716
Epoch 8/15
543/543 [==============================] - 116s 214ms/step - loss: 0.0638 - acc: 0.9768
Epoch 9/15
543/543 [==============================] - 117s 215ms/step - loss: 0.0529 - acc: 0.9801
Epoch 10/15
543/543 [==============================] - 116s 214ms/step - loss: 0.0426 - acc: 0.9848
Epoch 11/15
543/543 [==============================] - 116s 213ms/step - loss: 0.0516 - acc: 0.9814
Epoch 12/15
480/543 [=========================>....] - ETA: 13s - loss: 0.0339 - acc: 0.9875

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [18]:
for i, r in enumerate(exp2b_history):
    model_file = "exp2_prev-page_repeat-%02d.hdf5" % (i,)
    print(str(i) + ' ' + str(r['kappa']) + ' ' + str(r['accuracy']) + ' ' + str(r['f1_macro']) + ' ' + model_file)

0 0.5743994042608302 0.8987242394504417 0.7871291265380997 exp2_img_repeat-00.hdf5
1 0.5825763430506814 0.9048086359175662 0.7909664021046532 exp2_img_repeat-01.hdf5
2 0.5725330423965249 0.8977428851815505 0.7862144503840616 exp2_img_repeat-02.hdf5
3 0.5900362900440113 0.9028459273797841 0.794934126290217 exp2_img_repeat-03.hdf5
4 0.5781170523593033 0.8969578017664377 0.789052520806111 exp2_img_repeat-04.hdf5
5 0.5960922016537417 0.8985279685966634 0.7980312200658954 exp2_img_repeat-05.hdf5
6 0.5692776231180224 0.8985279685966634 0.7845232260978263 exp2_img_repeat-06.hdf5
7 0.5853219844055793 0.8971540726202158 0.7926599616536878 exp2_img_repeat-07.hdf5
8 0.5805844652202035 0.9038272816486752 0.7900121994875273 exp2_img_repeat-08.hdf5
9 0.5735645039475368 0.9004906771344455 0.7866165317761218 exp2_img_repeat-09.hdf5


In [ ]:
for i, r in enumerate(exp2b_history):
    model_file = "exp2_prev-page_repeat-%02d.hdf5" % (i,)
    print(str(i) + ' ' + str(r['kappa']) + ' ' + str(r['accuracy']) + ' ' + model_file)

In [ ]:
# load best model
model.load_weights("exp2_prev-page_repeat-03.hdf5.hdf5")

y_predict = model.predict_generator(TextFeatureGenerator2(data_test, batch_size=256)).argmax(axis=-1)
y_true = [label2Idx[x[1]] for x in data_test]
print("Accuracy: " + str(sklm.accuracy_score(y_true, y_predict)))
print("Kappa: " + str(sklm.cohen_kappa_score(y_true, y_predict)))